## **Reading data from bronze layer**

In [0]:
df = spark.read.csv('abfss://salesdata@adlssouthindiastorage.dfs.core.windows.net/broze/', header=True, inferSchema=True)
display(df)

## **masking delear_id**

In [0]:
from pyspark.sql.functions import substring, concat, col, lit

df2 = df.withColumn( \
    'masking_dealer_id', \
    concat( \
        substring(col('Dealer_id'), 1, 3), \
        lit('***'), \
        substring(col('Dealer_id'), 6, 2  ) \
         ) \
)

display(df2.limit(5))
                    


## **spliting model ID**

In [0]:
from pyspark.sql.functions import split

df3 = df2.withColumn( \
    'Model', split(
        col('Model_ID'), '-'
    )[0]
)

display(df3.limit(10))

## **Dropping the deplicate row from the table**

In [0]:
# checking for duplicates in the dataset if there we can write in saparete folder
print("Number of rows in the DataFrame df3: " + str(df3.count()))

duplicate = df3.groupBy(df3.columns).count().filter(col('count') > 1)

print("Number of rows in the DataFrame duplicate: " + str(duplicate.count()))

display(duplicate)

In [0]:
df4 = df3.dropDuplicates()

print("Number of rows in the DataFrame: " + str(df4.count()))

display(df4.limit(10))

## **Revenue per unit sold**

In [0]:
df5 = df4.withColumn(
    'Revenue_per_unit_sold', col('Revenue') / col('Units_Sold')
    )

display(df5.limit(10))

In [0]:
from pyspark.sql.functions import avg

df5.groupBy('year', 'branchname').agg(
    avg('Revenue_per_unit_sold').alias('avg_revenue_per_unit_sold')
).orderBy('year', 'branchname', ascending=[1, 0]).display()


Databricks visualization. Run in Databricks to view.

## **writing to gold layer**

In [0]:
df5.write.format('parquet').mode('overwrite').option('path', 'abfss://salesdata@adlssouthindiastorage.dfs.core.windows.net/silver/').saveAsTable('ansh_lamba.salesdata.silver')

print("transferred data loaded into silver layer")

In [0]:
%sql
select count(*) from parquet.`abfss://salesdata@adlssouthindiastorage.dfs.core.windows.net/silver/`;

In [0]:
%sql
select * from ansh_lamba.salesdata.silver limit 5;

In [0]:
%sql
drop table ansh_lamba.salesdata.silver